In [1]:
import pyrender
import json
import os
from os.path import join
import pickle
from pysdf import SDF
import trimesh
import numpy as np
from tqdm import *
import cv2
import torch
from manotorch.manolayer import ManoLayer
os.environ['MUJOCO_GL'] = 'osmesa'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'
def vis_smpl(out_path, image, nf, vertices, faces, camera_R, camera_T, K = np.array([1031.8450927734375, 0.0, 932.1596069335938, 0.0, 1022.4588623046875, 541.9437255859375, 0.0, 0.0, 1.0]).reshape((3,3))):
    # outname = os.path.join(out_path, '{:06d}.jpg'.format(nf))
    # if os.path.exists(outname): return
    render_data = {}
    assert vertices.shape[1] == 3 and len(vertices.shape) == 2, 'shape {} != (N, 3)'.format(vertices.shape)
    render_data = {"vertices": vertices, "faces": faces, "vid":0, "name": "human_{}_0".format(nf)}

    camera = {"K": K,
        "R": camera_R,
        "T":camera_T}
    from visualize.renderer_raw import Renderer
    render = Renderer(height=3840, width=2160, faces=None)
    image_vis, depth = render.render(render_data, camera, image, add_back=True)
    # print(depth)
    if nf=='gt':
        outname = os.path.join(out_path, nf+'.jpg')
    else: 
        outname = os.path.join(out_path, nf.zfill(4)+'.jpg')
    cv2.imwrite(outname, image_vis)
    return image_vis, depth 

/root/anaconda3/envs/render/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
contactgen_path = 'final_results/contactgen_500_2000'
seqs = os.listdir(contactgen_path)
for seq in seqs:
    seq_path = join(contactgen_path,seq)
    res_path = join(seq_path, 'results')
    break

# opt 不优化R

In [20]:
calib_path = "/root/code/seqs/calibration_all.json"
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
view = '1'
camera_pose = np.vstack((np.asarray(calib_dome[str(1)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(1)]['K']).reshape((3,3))

manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
hand_faces = manolayer.th_faces

In [16]:
datapath = '/root/code/seqs/0303_data/'
obj_path = '/root/code/seqs/object/'
path = 'optim/val_gaze/0071.npy'
# path = 'optim/val_gaze/0071_no_opt_R.npy'
res = np.load((path),allow_pickle=True).item()
print(res.keys())
pred_motion = torch.tensor(res['hand_motion']).cpu()
obj_pose = torch.tensor(res['obj_motion']).cpu().float()
seq = res['seq']
print(seq)
seq_path = join(datapath,seq)

meta_path = join(seq_path,'meta.pkl')
with open(meta_path,'rb')as f:
    meta = pickle.load(f)
    
active_obj = meta['active_obj']
obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
obj_mesh = trimesh.load(obj_mesh_path)
obj_verts = obj_mesh.vertices
obj_faces = obj_mesh.faces

seq_len = obj_pose.shape[0]
obj_verts = torch.tensor(obj_verts).unsqueeze(0).repeat(seq_len,1,1).float()


pred_trans = pred_motion[:,:3]
pred_theta = pred_motion[:,3:51]
pred_rot = pred_motion[:,3:6]
length = pred_motion.shape[0]


hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
mano_beta = hand_params[0,51:].unsqueeze(0).repeat(seq_len,1)

pred_output = manolayer(pred_theta, mano_beta)
pred_verts = pred_output.verts  + pred_trans.unsqueeze(1)
# pred_verts = pred_output.verts - pred_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)


obj_R = obj_pose[:,:3,:3]
obj_R = torch.einsum('...ij->...ji', [obj_R])
obj_T = obj_pose[:,:3,3].unsqueeze(1)

obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T

# print(obj_verts.shape,mano_verts.shape)
render_path = path.replace('.npy','render_all')
print(render_path)
render_out = join(render_path,seq,str(view))
os.makedirs(render_out,exist_ok=True)
for k in range(len(pred_verts)):
    img = np.zeros((2160, 3840,3), np.uint8) + 255
    # print(k)
    pred_all_verts = np.vstack((pred_verts[k].numpy(),obj_verts[k].numpy()))
    faces = np.vstack((hand_faces,obj_faces+778))
    pred_img, _ = vis_smpl(render_out, img, str(k), pred_all_verts, faces,camera_pose[:3,:3],camera_pose[:3,3],K)
    # outname = os.path.join(render_out, '{:06d}.jpg'.format(k))
    # cv2.imwrite(outname, pred_img)

dict_keys(['hand_motion', 'obj_motion', 'seq'])
0071
optim/val_gaze/0071render_all


/tmp/ipykernel_3356364/3909446944.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obj_pose = torch.tensor(res['obj_motion']).cpu().float()


In [17]:
datapath = '/root/code/seqs/gazehoi_list_test_0303.txt'
with open(datapath,'r') as f:
    info_list = f.readlines()
test_seqs = []
for info in info_list:
    seq = info.strip()
    test_seqs.append(seq)

In [ ]:
# test_seqs 批量

datapath = '/root/code/seqs/0303_data/'
obj_path = '/root/code/seqs/object/'
for seq in tqdm(test_seqs):
    path = f'optim/val_gaze/{seq}.npy'
    # path = 'optim/val_gaze/0071_no_opt_R.npy'
    res = np.load((path),allow_pickle=True).item()
    print(res.keys())
    pred_motion = torch.tensor(res['hand_motion']).cpu()
    obj_pose = torch.tensor(res['obj_motion']).cpu().float()
    seq = res['seq']
    print(seq)
    seq_path = join(datapath,seq)

    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
        
    active_obj = meta['active_obj']
    obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
    obj_mesh = trimesh.load(obj_mesh_path)
    obj_verts = obj_mesh.vertices
    obj_faces = obj_mesh.faces

    seq_len = obj_pose.shape[0]
    obj_verts = torch.tensor(obj_verts).unsqueeze(0).repeat(seq_len,1,1).float()


    pred_trans = pred_motion[:,:3]
    pred_theta = pred_motion[:,3:51]
    pred_rot = pred_motion[:,3:6]
    length = pred_motion.shape[0]


    hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
    mano_beta = hand_params[0,51:].unsqueeze(0).repeat(seq_len,1)

    pred_output = manolayer(pred_theta, mano_beta)
    pred_verts = pred_output.verts  + pred_trans.unsqueeze(1)
    # pred_verts = pred_output.verts - pred_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)


    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)

    obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T

    # print(obj_verts.shape,mano_verts.shape)
    render_path = path.replace('.npy','render_all')
    print(render_path)
    render_out = join(render_path,seq,str(view))
    os.makedirs(render_out,exist_ok=True)
    for k in range(len(pred_verts)):
        img = np.zeros((2160, 3840,3), np.uint8) + 255
        # print(k)
        pred_all_verts = np.vstack((pred_verts[k].numpy(),obj_verts[k].numpy()))
        faces = np.vstack((hand_faces,obj_faces+778))
        pred_img, _ = vis_smpl(render_out, img, str(k), pred_all_verts, faces,camera_pose[:3,:3],camera_pose[:3,3],K)
        # outname = os.path.join(render_out, '{:06d}.jpg'.format(k))
        # cv2.imwrite(outname, pred_img)

In [22]:
stage1_ok_seq = ['0009',
                '0011',
                '0038',
                '0071',
                '0198',
                '0260',
                '0390',
                '0412',
                '0413',
                '0421',
                '0425',
                '0436',
                '0522',
                '0535',
                '0536',
                '0569',
                '0644',
                '0670',
                '0681',
                '0746',
                '0773',
                '0796',
                '0808',
                '0827',
                '0957',
                '1004',
                '1009',
                '1020',
                '1059',
                '1246',
                '1282',
                '1294',
                '1299',
                '1306',
                '1313',
                '1315',
                '1317',
                '1352'
                ]

In [23]:
len(stage1_ok_seq)

38

In [24]:
stage2_ok_seq = ['0009',
                '0038',
                '0071', # view3 ok
                '0260',
                '0436',
                '0522',
                '0681',
                '0746',
                '1059',
                '1246', # s2前半部分ok
                '1306',
                '1315' # s2前半部分ok
                ]

In [ ]:
stage2_ok_seq = ['0009', # view8 ok
                '0038',
                '0071', # view8 ok
                '0260', # view8 ok
                '0436', # view8 ok
                '0522',
                '0681', # view8 ok
                '0746',
                '1246', # s2前半部分ok
                '1306',
                '1315' # s2前半部分ok # view8 ok
                ]

new 0312_val_gaze_align

In [ ]:
stage1_ok = ['0009',
             '0014',
             '',]

In [ ]:
stage1_mianqiang_ok = ['0071']

In [ ]:
stage1_ok_opt_chuanmo = ['0013']

## 用手物不同颜色把ours结果可视化出来

### 一个物体

In [13]:
def vis_smpl_color(out_path, image, nf, vertices, faces, camera_R, camera_T, K,index=778,face_index=1538):
    outname = os.path.join(out_path, '{:06d}.jpg'.format(nf))
    # if os.path.exists(outname): return
    render_data = {}
    assert vertices.shape[1] == 3 and len(vertices.shape) == 2, 'shape {} != (N, 3)'.format(vertices.shape)
    # render_data = {"vertices": vertices, "faces": faces, "vid":0, "name": "human_{}_0".format(nf)}
    render_data_hand = {"vertices": vertices[:index], "faces": faces[:face_index], "vid":0, "name": "human_{}_0_h".format(nf),"col_code":(60,100,160)}
    render_data_obj = {"vertices": vertices[index:], "faces": faces[face_index:]-778, "vid":0, "name": "human_{}_0_o".format(nf),"col_code":(180, 140, 40)}
    #(112,128,144)
    camera = {"K": K,
        "R": camera_R,
        "T":camera_T}
    from visualize.renderer import Renderer
    render = Renderer(height=3840, width=2160, faces=None)
    image_vis, depth = render.render([render_data_hand,render_data_obj], camera, image, add_back=True)
    # print(depth)
    # outname = os.path.join(out_path, '{:06d}.jpg'.format(nf))
    # cv2.imwrite(outname, image_vis)
    return image_vis, depth  

def img2video(image_path, video_path):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 用于mp4格式的生成
    height = 2160
    weight = 3840
    fps = 10
    videowriter = cv2.VideoWriter(video_path, fourcc, fps, (weight, height))  # 创建一个写入视频对象

    imgs = sorted(os.listdir(image_path))
    for img in tqdm(imgs):
        frame = cv2.imread(join(image_path, img))
        videowriter.write(frame)
    videowriter.release()


datapath = '/root/code/seqs/0303_data/'
res_path = 'final_results/check_vis'
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
hand_faces = manolayer.th_faces
print(hand_faces.shape)

calib_path = "/root/code/seqs/calibration_all.json"
obj_path = '/root/code/seqs/object/'
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
view = 1
camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))

# seqs = ['0009','0071','0155','0327','0553','0641','0664','1016','1327','1354']
seqs = ['1016','0641']
for seq in seqs:
    # break
    pred_path = join(res_path,seq,f'{seq}.npy')
    res = np.load((pred_path),allow_pickle=True).item()
    pred_motion = torch.tensor(res['hand_motion']).cpu()
    obj_pose = torch.tensor(res['obj_motion']).cpu().float()

    seq_path = join(datapath,seq)

    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
            meta = pickle.load(f)
        
    active_obj = meta['active_obj']
    obj_name_list = meta['obj_name_list']
    print(obj_name_list)
    obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
    obj_mesh = trimesh.load(obj_mesh_path)
    obj_verts = obj_mesh.vertices
    obj_faces = obj_mesh.faces
    
    seq_len = obj_pose.shape[0]
    obj_verts = torch.tensor(obj_verts).unsqueeze(0).repeat(seq_len,1,1).float()
    
    
    pred_trans = pred_motion[:,:3]
    pred_theta = pred_motion[:,3:51]
    pred_rot = pred_motion[:,3:6]
    length = pred_motion.shape[0]

    hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
    mano_beta = hand_params[0,51:].unsqueeze(0).repeat(seq_len,1)

    pred_output = manolayer(pred_theta, mano_beta)
    pred_verts = pred_output.verts - pred_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)


    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    
    obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T

    render_path = pred_path.replace('.npy','render_color')
    render_out = join(render_path,seq,str(view))
    os.makedirs(render_out,exist_ok=True)
    for k in tqdm(range(len(pred_verts))):
        img = np.zeros((2160, 3840,3), np.uint8) + 255
        # print(k)
        pred_all_verts = np.vstack((pred_verts[k].numpy(),obj_verts[k].numpy()))
        faces = np.vstack((hand_faces,obj_faces+778))
        pred_img, _ = vis_smpl_color(render_out, img, k, pred_all_verts, faces,camera_pose[:3,:3],camera_pose[:3,3],K)
        outname = os.path.join(render_out, '{:06d}.jpg'.format(k))
        # print(outname)
        cv2.imwrite(outname, pred_img)
        # break
    # break
    video_name = 'seq' + seq + '_view' + str(view) + '.mp4' 
    video_path = join(render_path,video_name)
    img2video(render_out,video_path)
    # video_name = 'seq' + seq + '_view' + str(view) + '.mp4' 
    # video_path = join(render_path,'render_videos')
    # os.makedirs(video_path,exist_ok=True)
    # video_path = join(video_path,video_name)
    # img2video(render_out,video_path)
    # # seq_index += 1
    # # if render_num == seq_index:
    # #     break
    # # break





/tmp/ipykernel_311642/2401349127.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obj_pose = torch.tensor(res['obj_motion']).cpu().float()


torch.Size([1538, 3])
['003_vase_1', '003_vase_2', '003_vase_3', '003_vase_4']


100%|██████████| 163/163 [00:19<00:00,  8.57it/s]


['006_apple', '006_pear', '006_lemon']


100%|██████████| 160/160 [00:18<00:00,  8.80it/s]


### 多个物体

In [38]:

datapath = '/root/code/seqs/0303_data/'
res_path = 'final_results/check_vis'
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
hand_faces = manolayer.th_faces
print(hand_faces.shape)

calib_path = "/root/code/seqs/calibration_all.json"
obj_path = '/root/code/seqs/object/'
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
view = 1
camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))

seqs = ['1327']
# seqs = ['0009','0071','0155','0327','0553','0641','0664','1016','1327','1354']
for seq in seqs:
    # break
    seq_len = 195
    pred_path = join(res_path,seq,f'{seq}.npy')
    res = np.load((pred_path),allow_pickle=True).item()
    pred_motion = torch.tensor(res['hand_motion']).cpu()[:seq_len]
    obj_pose = torch.tensor(res['obj_motion']).cpu().float()[:seq_len]

    seq_path = join(datapath,seq)

    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
            meta = pickle.load(f)
        
    active_obj = meta['active_obj']
    obj_name_list = meta['obj_name_list']
    print(obj_name_list)

    obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
    obj_mesh = trimesh.load(obj_mesh_path)
    obj_verts = obj_mesh.vertices
    obj_faces = obj_mesh.faces
    
    # seq_len = obj_pose.shape[0]
    obj_verts = torch.tensor(obj_verts).unsqueeze(0).repeat(seq_len,1,1).float()
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    obj_verts = (torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T).numpy()

    for obj in obj_name_list:
        if obj == active_obj:
            continue
        obj_mesh_path = join(obj_path,obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_verts_ = obj_mesh.vertices
        obj_faces_ = obj_mesh.faces
        
        obj_verts_ = torch.tensor(obj_verts_).unsqueeze(0).repeat(seq_len,1,1).float()
        obj_pose_ = torch.tensor(np.load(join(seq_path,obj+'_pose_trans.npy'))).reshape(-1,3,4)[:seq_len].float()
        obj_R = obj_pose_[:,:3,:3]
        obj_R = torch.einsum('...ij->...ji', [obj_R])
        obj_T = obj_pose_[:,:3,3].unsqueeze(1)
        obj_verts_ = (torch.einsum('fpn,fnk->fpk',obj_verts_,obj_R) + obj_T).numpy()
        obj_faces = np.vstack((obj_faces,obj_faces_+obj_verts.shape[1]))
        obj_verts = np.concatenate((obj_verts,obj_verts_),axis=1)
    print(obj_verts.shape)
    
    pred_trans = pred_motion[:,:3]
    pred_theta = pred_motion[:,3:51]
    pred_rot = pred_motion[:,3:6]
    length = pred_motion.shape[0]

    hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
    mano_beta = hand_params[0,51:].unsqueeze(0).repeat(seq_len,1)

    pred_output = manolayer(pred_theta, mano_beta)
    pred_verts = pred_output.verts - pred_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)


    render_path = pred_path.replace('.npy','render_color_multi_obj')
    render_out = join(render_path,seq,str(view))
    os.makedirs(render_out,exist_ok=True)
    for k in tqdm(range(len(pred_verts))):
        img = np.zeros((2160, 3840,3), np.uint8) + 255
        # print(k)
        pred_all_verts = np.vstack((pred_verts[k].numpy(),obj_verts[k]))
        faces = np.vstack((hand_faces,obj_faces+778))
        pred_img, _ = vis_smpl_color(render_out, img, k, pred_all_verts, faces,camera_pose[:3,:3],camera_pose[:3,3],K)
        outname = os.path.join(render_out, '{:06d}.jpg'.format(k))
        # print(outname)
        cv2.imwrite(outname, pred_img)
        # break
    video_name = 'seq' + seq + '_view' + str(view) + '.mp4' 
    video_path = join(render_path,video_name)
    img2video(render_out,video_path)
    # break

torch.Size([1538, 3])
['006_banana', '006_apple', '002_plastic_furniture_top', '002_plastic_furniture_bottom']


/tmp/ipykernel_311642/3040301727.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obj_pose = torch.tensor(res['obj_motion']).cpu().float()[:seq_len]


(195, 43726, 3)


100%|██████████| 284/284 [00:32<00:00,  8.70it/s]


In [17]:
# seqs = ['0155','0327','1016','0641']
seqs = ['0009']
for seq in seqs:
    print(seq)
    render_path = f'final_results/check_vis/{seq}/{seq}render_color_multi_obj/{seq}/1/'
    video_name = 'seq_10fps' + seq + '_view' + str(view) + '.mp4' 
    video_path = join(f'final_results/check_vis/{seq}/',video_name)
    img2video(render_path,video_path)

0009


100%|██████████| 345/345 [00:38<00:00,  8.88it/s]


## 渲染g2ho和o2h的多物体带颜色结果

In [40]:

datapath = '/root/code/seqs/0303_data/'
# res_path = 'final_results/check_vis/g2ho'
res_path = 'final_results/check_vis/o2h'
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano',side='right')
hand_faces = manolayer.th_faces
print(hand_faces.shape)

calib_path = "/root/code/seqs/calibration_all.json"
obj_path = '/root/code/seqs/object/'
with open(calib_path) as f:
    calib_dome = json.load(f)
    f.close()
view = 1
camera_pose = np.vstack((np.asarray(calib_dome[str(view)]['RT']).reshape((3,4)), np.ones(4) ))
K = np.asarray(calib_dome[str(view)]['K']).reshape((3,3))

select_seqs = ['1327']
# path = 'save/0304_g2ho/samples_0304_g2ho_000150000_seed10_0307_test/results.npy'
path = 'save/0303_o2h/samples_0303_o2h_000150000_seed10_0307_test/results.npy'
# seqs = ['0009','0071','0155','0327','0553','0641','0664','1016','1327','1354']
res = np.load(path,allow_pickle=True).item()
all_hand_motions = torch.tensor(res['all_hand_motions']).cpu().float() # obj_motion (bs,nf,4,3,4)
all_obj_motions = torch.tensor(res['all_obj_motions']).cpu().float() # obj_motion (bs,nf,4,3,4)
seqs = res['seqs']
for seq in select_seqs:
    # break
    i = seqs.index(seq)
    # print(i)
    # print(seqs[i])
    # continue
    # pred_path = join(res_path,seq,f'{seq}.npy')
    # res = np.load((pred_path),allow_pickle=True).item()
    if seq == '0009':
        seq_len = 345
    if seq == '0155':
        seq_len = 150
    if seq == '0327':
        seq_len = 155
    if seq == '0641':
        seq_len = 90
    if seq == '1016':
        seq_len = 163
    if seq == '1313':
        seq_len = 80
    if seq == '1327':
        seq_len = 195
    pred_motion = all_hand_motions[i][:seq_len]
    obj_pose = all_obj_motions[i][:seq_len]
    print(pred_motion.shape,obj_pose.shape)

    seq_path = join(datapath,seq)

    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
            meta = pickle.load(f)
        
    active_obj = meta['active_obj']
    obj_name_list = meta['obj_name_list']
    print(obj_name_list)

    obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
    obj_mesh = trimesh.load(obj_mesh_path)
    obj_verts = obj_mesh.vertices
    obj_faces = obj_mesh.faces
    
    
    # seq_len = obj_pose.shape[0]
    obj_verts = torch.tensor(obj_verts).unsqueeze(0).repeat(seq_len,1,1).float()
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    obj_verts = (torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T).numpy()

    for obj in obj_name_list:
        if obj == active_obj:
            continue
        obj_mesh_path = join(obj_path,obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_verts_ = obj_mesh.vertices
        obj_faces_ = obj_mesh.faces
        
        obj_verts_ = torch.tensor(obj_verts_).unsqueeze(0).repeat(seq_len,1,1).float()
        obj_pose_ = torch.tensor(np.load(join(seq_path,obj+'_pose_trans.npy'))).reshape(-1,3,4)[0].repeat(seq_len,1,1).float()
        obj_R = obj_pose_[:,:3,:3]
        obj_R = torch.einsum('...ij->...ji', [obj_R])
        obj_T = obj_pose_[:,:3,3].unsqueeze(1)
        print(obj_verts_.shape,obj_pose_.shape)
        obj_verts_ = (torch.einsum('fpn,fnk->fpk',obj_verts_,obj_R) + obj_T).numpy()
        obj_faces = np.vstack((obj_faces,obj_faces_+obj_verts.shape[1]))
        obj_verts = np.concatenate((obj_verts,obj_verts_),axis=1)
    print(obj_verts.shape)
    
    pred_trans = pred_motion[:,:3]
    pred_theta = pred_motion[:,3:51]
    pred_rot = pred_motion[:,3:6]
    length = pred_motion.shape[0]

    hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
    mano_beta = hand_params[0,51:].unsqueeze(0).repeat(seq_len,1)

    pred_output = manolayer(pred_theta, mano_beta)
    pred_verts = pred_output.verts - pred_output.joints[:, 0].unsqueeze(1) + pred_trans.unsqueeze(1)


    render_out = join(res_path,seq,str(view))
    os.makedirs(render_out,exist_ok=True)
    for k in tqdm(range(len(pred_verts))):
        img = np.zeros((2160, 3840,3), np.uint8) + 255
        # print(k)
        pred_all_verts = np.vstack((pred_verts[k].numpy(),obj_verts[k]))
        faces = np.vstack((hand_faces,obj_faces+778))
        pred_img, _ = vis_smpl_color(render_out, img, k, pred_all_verts, faces,camera_pose[:3,:3],camera_pose[:3,3],K)
        outname = os.path.join(render_out, '{:06d}.jpg'.format(k))
        # print(outname)
        cv2.imwrite(outname, pred_img)
        # break
    video_name = 'seq' + seq + '_view' + str(view) + '.mp4' 
    video_path = join(render_out,video_name)
    img2video(render_out,video_path)
    
    # break

torch.Size([1538, 3])
torch.Size([195, 51]) torch.Size([195, 3, 4])
['006_banana', '006_apple', '002_plastic_furniture_top', '002_plastic_furniture_bottom']
torch.Size([195, 10002, 3]) torch.Size([195, 3, 4])
torch.Size([195, 10004, 3]) torch.Size([195, 3, 4])
torch.Size([195, 10008, 3]) torch.Size([195, 3, 4])
(195, 43726, 3)


100%|██████████| 196/196 [00:22<00:00,  8.77it/s]


In [32]:
video_name = 'seq' + seq + '_view' + str(view) + '.mp4' 
video_path = join(render_out,video_name)
img2video(render_out,video_path)

100%|██████████| 346/346 [00:39<00:00,  8.83it/s]
